In [157]:
import numpy as np 
import pandas as pd
import os
from catboost import CatBoostClassifier , Pool
from sklearn.model_selection import train_test_split
import string

In [173]:
df_train = pd.read_csv("train.csv",index_col=0,)
df_test = pd.read_csv("test.csv",index_col=0)

In [174]:
df_train.fillna(df_train.median(),inplace=True)
df_train.fillna("NaN",inplace=True)

df_test.fillna(df_train.median(),inplace=True)
df_test.fillna("NaN",inplace=True)

In [175]:
df_test.day.isna().sum()

0

In [176]:
df_train.day = df_train.day.astype("int")
df_test.day = df_test.day.astype("int")

df_train.month = df_train.month.astype("int")
df_test.month = df_test.month.astype("int")

In [177]:
model = CatBoostClassifier(learning_rate =0.01,
                           class_weights=[0.8,0.2], eval_metric='AUC', 
                           logging_level='Silent', task_type="GPU",
                          use_best_model=True)

In [178]:
X = df_train.drop(["target"],axis=1)
y = df_train.target
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7)
X.dtypes

bin_0    float64
bin_1    float64
bin_2    float64
bin_3     object
bin_4     object
nom_0     object
nom_1     object
nom_2     object
nom_3     object
nom_4     object
nom_5     object
nom_6     object
nom_7     object
nom_8     object
nom_9     object
ord_0    float64
ord_1     object
ord_2     object
ord_3     object
ord_4     object
ord_5     object
day        int64
month      int64
dtype: object

In [179]:
catagorical_features = np.where(X.dtypes != np.float)[0]

In [180]:
catagorical_features

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20,
       21, 22])

In [181]:
catagorical_features = np.append(catagorical_features, [21 ,22])

In [182]:
train = Pool(X_train,y_train,cat_features = catagorical_features)
test = Pool(X_test,y_test,cat_features = catagorical_features)

In [183]:
%%time
model.fit(train, eval_set=test)

CPU times: user 1min 20s, sys: 48.3 s, total: 2min 8s
Wall time: 1min 36s


In [187]:
from sklearn.metrics import roc_auc_score

In [188]:
preds = model.predict_proba(test)

In [189]:
roc_auc_score(y_test, preds[:, 1])

0.7830974464158381

In [190]:
out = Pool(df_test,cat_features = catagorical_features)

In [191]:
sub = pd.read_csv("sample_submission.csv")
preds= model.predict_proba(out)
sub["target"] = preds[:, 1]
sub.head()

,id,target
0,600000,0.044959
1,600001,0.078724
2,600002,0.038464
3,600003,0.045253
4,600004,0.047117


In [192]:
sub.to_csv("preds/cat.csv",index=False)

In [4]:
os.makedirs("preds",exist_ok=True)
